In [8]:
import os
import shutil
import yaml
import random
from pathlib import Path
from collections import defaultdict

class YOLODatasetMerger:
    def __init__(self, dataset1_path, dataset2_path, output_path, 
                 train_ratio=0.7, val_ratio=0.2, test_ratio=0.1, 
                 stratify_second_dataset=True, random_seed=42):
        self.dataset1_path = Path(dataset1_path)
        self.dataset2_path = Path(dataset2_path)
        self.output_path = Path(output_path)
        
        # Split ratios
        self.train_ratio = train_ratio
        self.val_ratio = val_ratio
        self.test_ratio = test_ratio
        self.stratify_second_dataset = stratify_second_dataset
        
        # Set random seed for reproducibility
        random.seed(random_seed)
        
        # Validate split ratios
        if abs(train_ratio + val_ratio + test_ratio - 1.0) > 0.001:
            raise ValueError("Split ratios must sum to 1.0")
        
        # Load data.yaml files if they exist
        self.data1 = self.load_yaml_safe(self.dataset1_path / "data.yml")
        self.data2 = self.load_yaml_safe(self.dataset2_path / "data.yml")
        
    def load_yaml_safe(self, yaml_path):
        """Load YAML configuration file safely"""
        if yaml_path.exists():
            with open(yaml_path, 'r') as file:
                return yaml.safe_load(file)
        else:
            print(f"Warning: {yaml_path} not found, using defaults")
            return {'names': ['object'], 'nc': 1}
    
    def create_output_structure(self):
        """Create output directory structure: split/images and split/labels"""
        splits = ['train', 'val', 'test']
        
        for split in splits:
            (self.output_path / split / 'images').mkdir(parents=True, exist_ok=True)
            (self.output_path / split / 'labels').mkdir(parents=True, exist_ok=True)
    
    def detect_dataset_structure(self, dataset_path):
        """Detect the structure of the dataset"""
        dataset_path = Path(dataset_path)
        
        # Check for different possible structures
        structures = {
            'images_labels_split': dataset_path / 'images' / 'train',
            'split_images_labels': dataset_path / 'train' / 'images', 
            'flat': dataset_path / 'images'
        }
        
        for structure_name, test_path in structures.items():
            if test_path.exists():
                print(f"Detected {structure_name} structure for {dataset_path.name}")
                return structure_name
        
        print(f"Could not detect structure for {dataset_path.name}")
        return None
    
    def get_all_images_from_dataset(self, dataset_path):
        """Get all images from a dataset, handling different structures"""
        images = []
        structure = self.detect_dataset_structure(dataset_path)
        
        if structure == 'images_labels_split':
            # Structure: dataset/images/train, dataset/labels/train
            splits = ['train', 'val', 'test']
            for split in splits:
                img_dir = dataset_path / 'images' / split
                label_dir = dataset_path / 'labels' / split
                if img_dir.exists():
                    images.extend(self.get_images_from_directory(img_dir, label_dir, split))
                    
        elif structure == 'split_images_labels':
            # Structure: dataset/train/images, dataset/train/labels
            splits = ['train', 'val', 'test']
            for split in splits:
                img_dir = dataset_path / split / 'images'
                label_dir = dataset_path / split / 'labels'
                if img_dir.exists():
                    images.extend(self.get_images_from_directory(img_dir, label_dir, split))
                    
        elif structure == 'flat':
            # Structure: dataset/images, dataset/labels (all in one folder)
            img_dir = dataset_path / 'images'
            label_dir = dataset_path / 'labels'
            if img_dir.exists():
                images.extend(self.get_images_from_directory(img_dir, label_dir, 'all'))
        else:
            print(f"Warning: Could not find images in {dataset_path}")
            
        return images
    
    def get_images_from_directory(self, img_dir, label_dir, split_name):
        """Get images and corresponding labels from a directory"""
        images = []
        
        for img_file in img_dir.glob('*'):
            if img_file.suffix.lower() in ['.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.tif']:
                # Look for corresponding label
                label_file = label_dir / f"{img_file.stem}.txt"
                
                images.append({
                    'image_path': img_file,
                    'label_path': label_file if label_file.exists() else None,
                    'original_split': split_name
                })
                
        return images
    
    def stratified_split(self, images_list, dataset_name):
        """Create stratified splits ensuring representative distribution"""
        total_images = len(images_list)
        
        if total_images == 0:
            return {'train': [], 'val': [], 'test': []}
        
        # Calculate split sizes
        train_size = max(1, int(total_images * self.train_ratio))
        val_size = max(1, int(total_images * self.val_ratio)) if total_images > 1 else 0
        test_size = total_images - train_size - val_size
        
        # Ensure we don't have negative test size
        if test_size < 0:
            val_size += test_size
            test_size = 0
            
        # For very small datasets, ensure at least some images in train
        if train_size == 0:
            train_size = 1
            val_size = max(0, val_size - 1)
        
        # Shuffle the images for random distribution
        shuffled_images = images_list.copy()
        random.shuffle(shuffled_images)
        
        # Create splits
        splits = {
            'train': shuffled_images[:train_size],
            'val': shuffled_images[train_size:train_size + val_size],
            'test': shuffled_images[train_size + val_size:] if test_size > 0 else []
        }
        
        print(f"\n{dataset_name} stratified split:")
        print(f"  Total images: {total_images}")
        print(f"  Train: {len(splits['train'])} images ({len(splits['train'])/total_images:.1%})")
        print(f"  Val: {len(splits['val'])} images ({len(splits['val'])/total_images:.1%})")
        print(f"  Test: {len(splits['test'])} images ({len(splits['test'])/total_images:.1%})")
        
        return splits
    
    def copy_images_to_split(self, image_data_list, target_split, dataset_prefix):
        """Copy images and labels to the target split directory"""
        dst_images = self.output_path / target_split / 'images'
        dst_labels = self.output_path / target_split / 'labels'
        
        for img_data in image_data_list:
            img_path = img_data['image_path']
            label_path = img_data['label_path']
            
            # Create new filename with dataset prefix to avoid conflicts
            new_img_name = f"{dataset_prefix}_{img_path.name}"
            new_img_path = dst_images / new_img_name
            
            # Copy image
            shutil.copy2(img_path, new_img_path)
            
            # Copy corresponding label if it exists
            if label_path and label_path.exists():
                new_label_path = dst_labels / f"{Path(new_img_name).stem}.txt"
                shutil.copy2(label_path, new_label_path)
                print(f"  Copied: {img_path.name} -> {target_split}/{new_img_name}")
            else:
                print(f"  Copied: {img_path.name} -> {target_split}/{new_img_name} (no label)")
    
    def create_merged_yaml(self):
        """Create merged data.yaml file"""
        # Get class names from the datasets
        class_names = self.data1.get('names', ['object'])
        if isinstance(class_names, dict):
            # Convert dict format {0: "class1", 1: "class2"} to list
            class_names = [class_names[i] for i in sorted(class_names.keys())]
        
        merged_data = {
            'path': str(self.output_path.absolute()),
            'train': 'train/images',
            'val': 'val/images', 
            'test': 'test/images',
            'nc': len(class_names),
            'names': class_names
        }
        
        # Save merged data.yaml
        yaml_path = self.output_path / 'data.yaml'
        with open(yaml_path, 'w') as f:
            yaml.dump(merged_data, f, default_flow_style=False)
        
        print(f"\nCreated merged data.yaml with {merged_data['nc']} class(es): {class_names}")
    
    def merge_datasets(self):
        """Main method to merge datasets with stratified splitting"""
        print("Starting dataset merge with stratified splitting...")
        print(f"Dataset 1 path: {self.dataset1_path}")
        print(f"Dataset 2 path: {self.dataset2_path}")
        print(f"Output path: {self.output_path}")
        
        # Create output directory structure
        self.create_output_structure()
        
        # Get all images from both datasets
        images1 = self.get_all_images_from_dataset(self.dataset1_path)
        images2 = self.get_all_images_from_dataset(self.dataset2_path)
        
        print(f"\nFound {len(images1)} images in Dataset 1")
        print(f"Found {len(images2)} images in Dataset 2")
        
        if len(images1) == 0 and len(images2) == 0:
            print("\nERROR: No images found in either dataset!")
            print("Please check your dataset paths and structures.")
            return
        
        # Create stratified splits for both datasets
        if self.stratify_second_dataset and len(images2) > 0:
            # Stratify the smaller, more representative dataset
            splits2 = self.stratified_split(images2, "Dataset 2 (Representative)")
            
            # Stratify the larger dataset too for consistency
            splits1 = self.stratified_split(images1, "Dataset 1 (Large)")
        else:
            # Use existing splits if stratification is disabled
            print("Using existing splits (stratification disabled)")
            splits1 = self.get_existing_splits(images1)
            splits2 = self.get_existing_splits(images2)
        
        # Copy images to their assigned splits
        print("\nCopying images to merged dataset...")
        for split in ['train', 'val', 'test']:
            if splits1[split]:
                self.copy_images_to_split(splits1[split], split, "ds1")
            if splits2[split]:
                self.copy_images_to_split(splits2[split], split, "ds2")
        
        # Create merged data.yaml
        self.create_merged_yaml()
        
        # Print final statistics
        self.print_final_stats()
        
        print(f"\nDataset merge completed! Output saved to: {self.output_path}")
    
    def get_existing_splits(self, images_list):
        """Group images by their existing splits"""
        splits = {'train': [], 'val': [], 'test': []}
        for img_data in images_list:
            original_split = img_data['original_split']
            if original_split == 'all':
                # If flat structure, distribute randomly
                return self.stratified_split(images_list, "Dataset (flat structure)")
            else:
                splits[original_split].append(img_data)
        return splits
    
    def print_final_stats(self):
        """Print final statistics about the merged dataset"""
        print("\n" + "="*50)
        print("FINAL MERGED DATASET STATISTICS")
        print("="*50)
        
        total_images = 0
        total_labels = 0
        
        for split in ['train', 'val', 'test']:
            img_path = self.output_path / split / 'images'
            label_path = self.output_path / split / 'labels'
            
            if img_path.exists():
                images = list(img_path.glob('*'))
                labels = list(label_path.glob('*.txt')) if label_path.exists() else []
                
                # Count images from each dataset
                ds1_images = len([img for img in images if img.name.startswith('ds1_')])
                ds2_images = len([img for img in images if img.name.startswith('ds2_')])
                
                total_images += len(images)
                total_labels += len(labels)
                
                print(f"{split.upper()}:")
                print(f"  Total: {len(images)} images, {len(labels)} labels")
                print(f"  Dataset 1: {ds1_images} images")
                print(f"  Dataset 2: {ds2_images} images")
                if len(images) > 0:
                    print(f"  Dataset 2 representation: {ds2_images/len(images):.1%}")
                print()
        
        print(f"OVERALL TOTAL: {total_images} images, {total_labels} labels")


In [9]:
merger = YOLODatasetMerger(
    dataset1_path="./doors_500_yolov8/",
    dataset2_path="./trunk_tools_yolov8/", 
    output_path="./merged_dataset_v2",
    train_ratio=0.75,       # 75% training (16 representative images)
    val_ratio=0.15,         # 15% validation (3 representative images)
    test_ratio=0.10,        # 10% testing (2 representative images)
    stratify_second_dataset=True,
    random_seed=42
)
merger.merge_datasets()

Starting dataset merge with stratified splitting...
Dataset 1 path: doors_500_yolov8
Dataset 2 path: trunk_tools_yolov8
Output path: merged_dataset
Detected split_images_labels structure for doors_500_yolov8
Detected split_images_labels structure for trunk_tools_yolov8

Found 960 images in Dataset 1
Found 21 images in Dataset 2

Dataset 2 (Representative) stratified split:
  Total images: 21
  Train: 15 images (71.4%)
  Val: 3 images (14.3%)
  Test: 3 images (14.3%)

Dataset 1 (Large) stratified split:
  Total images: 960
  Train: 720 images (75.0%)
  Val: 144 images (15.0%)
  Test: 96 images (10.0%)

Copying images to merged dataset...
  Copied: 777_png.rf.7a675d633ea2d5b5818ea173c0d6c886.jpg -> train/ds1_777_png.rf.7a675d633ea2d5b5818ea173c0d6c886.jpg
  Copied: 651_png.rf.38c294b0157c364d164904a1b3e88e1a.jpg -> train/ds1_651_png.rf.38c294b0157c364d164904a1b3e88e1a.jpg
  Copied: 1107_png.rf.eefcd326588001598fbaa5839c790cf1.jpg -> train/ds1_1107_png.rf.eefcd326588001598fbaa5839c790cf1.

  Copied: 1121_png.rf.f16dc75001ba244cf4d18515dc627c91.jpg -> train/ds1_1121_png.rf.f16dc75001ba244cf4d18515dc627c91.jpg
  Copied: 966_png.rf.2f09dd5ec9d7a890ffb91654b0b737a9.jpg -> train/ds1_966_png.rf.2f09dd5ec9d7a890ffb91654b0b737a9.jpg
  Copied: 1051_png.rf.bdf9bfb6f22c1de49079ac2fd008b6f9.jpg -> train/ds1_1051_png.rf.bdf9bfb6f22c1de49079ac2fd008b6f9.jpg
  Copied: 703_png.rf.46e1dc13dc5cfdb799a60853457c444b.jpg -> train/ds1_703_png.rf.46e1dc13dc5cfdb799a60853457c444b.jpg
  Copied: 1088_png.rf.11f340d5afdd0a738af477faeecb2e1e.jpg -> train/ds1_1088_png.rf.11f340d5afdd0a738af477faeecb2e1e.jpg
  Copied: 836_png.rf.c061353a768d6219eed59a7a6f345f65.jpg -> train/ds1_836_png.rf.c061353a768d6219eed59a7a6f345f65.jpg
  Copied: 931_png.rf.0bff2d50a544424685e60c1218ec162e.jpg -> train/ds1_931_png.rf.0bff2d50a544424685e60c1218ec162e.jpg
  Copied: 1015_png.rf.1a2b77850eddd5c70d49d8a98fcd62cb.jpg -> train/ds1_1015_png.rf.1a2b77850eddd5c70d49d8a98fcd62cb.jpg
  Copied: 891_png.rf.fe9ea05f4e1628b8ece

  Copied: 1064_png.rf.cb85b21bdd939a4ce0be8d5bdeed1b11.jpg -> train/ds1_1064_png.rf.cb85b21bdd939a4ce0be8d5bdeed1b11.jpg
  Copied: 1112_png.rf.830005d53358f201128d09b3ff7dae09.jpg -> train/ds1_1112_png.rf.830005d53358f201128d09b3ff7dae09.jpg
  Copied: 738_png.rf.0a1388eee597c26ad585fa707d2c5d10.jpg -> train/ds1_738_png.rf.0a1388eee597c26ad585fa707d2c5d10.jpg
  Copied: 945_png.rf.469c96c9cf4ffba4e9958496487bbfa5.jpg -> train/ds1_945_png.rf.469c96c9cf4ffba4e9958496487bbfa5.jpg
  Copied: 828_png.rf.259eac403aad0320b2e0c10b46168f7e.jpg -> train/ds1_828_png.rf.259eac403aad0320b2e0c10b46168f7e.jpg
  Copied: 942_png.rf.a7b50bc3eef89ef955103a64523898d5.jpg -> train/ds1_942_png.rf.a7b50bc3eef89ef955103a64523898d5.jpg
  Copied: 814_png.rf.bf624a465ac096837d3065b5c4f7cda8.jpg -> train/ds1_814_png.rf.bf624a465ac096837d3065b5c4f7cda8.jpg
  Copied: 1075_png.rf.00406f5b9dd021ae709a57f6d8da149d.jpg -> train/ds1_1075_png.rf.00406f5b9dd021ae709a57f6d8da149d.jpg
  Copied: 861_png.rf.9bb63f3aa71c7337bb794

  Copied: 729_png.rf.6f1520db8b42b4b1b8864ceb85bc154c.jpg -> train/ds1_729_png.rf.6f1520db8b42b4b1b8864ceb85bc154c.jpg
  Copied: 853_png.rf.776961f6f3bcf4df69a61ce2fa2a5c52.jpg -> train/ds1_853_png.rf.776961f6f3bcf4df69a61ce2fa2a5c52.jpg
  Copied: 1078_png.rf.6149f3d5a2088e82fc1885b42962dfe8.jpg -> train/ds1_1078_png.rf.6149f3d5a2088e82fc1885b42962dfe8.jpg
  Copied: 1108_png.rf.7576cbe411a875d6f0fe8f467e47822d.jpg -> train/ds1_1108_png.rf.7576cbe411a875d6f0fe8f467e47822d.jpg
  Copied: 1001_png.rf.58a1baf7a4b92feb95314a36969cea13.jpg -> train/ds1_1001_png.rf.58a1baf7a4b92feb95314a36969cea13.jpg
  Copied: 945_png.rf.e75c1b3084b808a7391fa26f80f3dbb9.jpg -> train/ds1_945_png.rf.e75c1b3084b808a7391fa26f80f3dbb9.jpg
  Copied: 1011_png.rf.ea60bdfe2012bd2f425a62649266ecf7.jpg -> train/ds1_1011_png.rf.ea60bdfe2012bd2f425a62649266ecf7.jpg
  Copied: 1119_png.rf.0dd3e4ebbaadb6c51f27489879989584.jpg -> train/ds1_1119_png.rf.0dd3e4ebbaadb6c51f27489879989584.jpg
  Copied: 1038_png.rf.81418a3ff6f2464a

  Copied: 1052_png.rf.79021123ff25704b9cd8b028882faea0.jpg -> train/ds1_1052_png.rf.79021123ff25704b9cd8b028882faea0.jpg
  Copied: 804_png.rf.ee4e4758dd74c360d799087cafd4f1cd.jpg -> train/ds1_804_png.rf.ee4e4758dd74c360d799087cafd4f1cd.jpg
  Copied: 1078_png.rf.85dd8aa0bacd66b74f05dde6ef1d1f6a.jpg -> train/ds1_1078_png.rf.85dd8aa0bacd66b74f05dde6ef1d1f6a.jpg
  Copied: 1012_png.rf.94b1b40f8d6d09a7131d39cc7a765fe1.jpg -> train/ds1_1012_png.rf.94b1b40f8d6d09a7131d39cc7a765fe1.jpg
  Copied: 725_png.rf.97b6af8126e1777616a3dee6f1e6d638.jpg -> train/ds1_725_png.rf.97b6af8126e1777616a3dee6f1e6d638.jpg
  Copied: 1024_png.rf.201456a31ee1c52c816e8a317982523b.jpg -> train/ds1_1024_png.rf.201456a31ee1c52c816e8a317982523b.jpg
  Copied: 921_png.rf.035bd0dcd680f606c7f490fbe943abe5.jpg -> train/ds1_921_png.rf.035bd0dcd680f606c7f490fbe943abe5.jpg
  Copied: 1078_png.rf.e3f3236abfc0eaacbb0c0a49791ffbb1.jpg -> train/ds1_1078_png.rf.e3f3236abfc0eaacbb0c0a49791ffbb1.jpg
  Copied: 859_png.rf.bbfbfb5f3649116cb

...

  Copied: 939_png.rf.cba56b9a0524ee6f8ed94f72a2d33972.jpg -> train/ds1_939_png.rf.cba56b9a0524ee6f8ed94f72a2d33972.jpg
  Copied: 1108_png.rf.57420eeca630fe69e1963b047c314617.jpg -> train/ds1_1108_png.rf.57420eeca630fe69e1963b047c314617.jpg
  Copied: 1049_png.rf.8e76a08cdb05bde09f4e4a283caf9a09.jpg -> train/ds1_1049_png.rf.8e76a08cdb05bde09f4e4a283caf9a09.jpg
  Copied: 793_png.rf.613a8849ab39bfcc6f1815550d9eee63.jpg -> train/ds1_793_png.rf.613a8849ab39bfcc6f1815550d9eee63.jpg
  Copied: 868_png.rf.4aa3f7dd24f80fe037bdef6ac47f894f.jpg -> train/ds1_868_png.rf.4aa3f7dd24f80fe037bdef6ac47f894f.jpg
  Copied: 963_png.rf.aebe3c9d88580872c37e860c15f3afcd.jpg -> train/ds1_963_png.rf.aebe3c9d88580872c37e860c15f3afcd.jpg
  Copied: 949_png.rf.69697f1a1c408b3d6bfcf54775deabce.jpg -> train/ds1_949_png.rf.69697f1a1c408b3d6bfcf54775deabce.jpg
  Copied: 788_png.rf.a2c5b9c7e8160c833c476cfcb97fc6a1.jpg -> train/ds1_788_png.rf.a2c5b9c7e8160c833c476cfcb97fc6a1.jpg
  Copied: 776_png.rf.e624ad7f942781a7b2cdba0

  Copied: 942_png.rf.0b673f26d28d7a84b3f158b32d5cfad2.jpg -> train/ds1_942_png.rf.0b673f26d28d7a84b3f158b32d5cfad2.jpg
  Copied: 906_png.rf.7885250b40f7e82701d06cdb83dcf770.jpg -> train/ds1_906_png.rf.7885250b40f7e82701d06cdb83dcf770.jpg
  Copied: 652_png.rf.05b04674ea36cbc84b7832504f734045.jpg -> train/ds1_652_png.rf.05b04674ea36cbc84b7832504f734045.jpg
  Copied: 1132_png.rf.1e01f5c53533bbfa914bd077efea3a09.jpg -> train/ds1_1132_png.rf.1e01f5c53533bbfa914bd077efea3a09.jpg
  Copied: 799_png.rf.7dee93bf34a30b111608ebbda2e8187c.jpg -> train/ds1_799_png.rf.7dee93bf34a30b111608ebbda2e8187c.jpg
  Copied: 920_png.rf.1f0ab6de3d66b8870ebfb975bdba2d2a.jpg -> train/ds1_920_png.rf.1f0ab6de3d66b8870ebfb975bdba2d2a.jpg
  Copied: 682_png.rf.f441207073c68a9d8668d29491dea33b.jpg -> train/ds1_682_png.rf.f441207073c68a9d8668d29491dea33b.jpg
  Copied: 658_png.rf.980dd61d62ce0f15126b41b4bda64a39.jpg -> train/ds1_658_png.rf.980dd61d62ce0f15126b41b4bda64a39.jpg
  Copied: 1042_png.rf.5a5152570af3d53a1c29ef1a

  Copied: 659_png.rf.371b1ae52d4dab36fa4140cb126aee16.jpg -> val/ds1_659_png.rf.371b1ae52d4dab36fa4140cb126aee16.jpg
  Copied: 875_png.rf.685b19054be98b833d089688a8c3423e.jpg -> val/ds1_875_png.rf.685b19054be98b833d089688a8c3423e.jpg
  Copied: 1064_png.rf.4888dae7089323f2cdd9991b7901336f.jpg -> val/ds1_1064_png.rf.4888dae7089323f2cdd9991b7901336f.jpg
  Copied: 793_png.rf.3e1d6078af716ee444ce675e5b1f616e.jpg -> val/ds1_793_png.rf.3e1d6078af716ee444ce675e5b1f616e.jpg
  Copied: 1104_png.rf.af9bd9bf357c88cc06fc67603b94548e.jpg -> val/ds1_1104_png.rf.af9bd9bf357c88cc06fc67603b94548e.jpg
  Copied: 956_png.rf.1789bf6a0064d29b627d33e372f82dd0.jpg -> val/ds1_956_png.rf.1789bf6a0064d29b627d33e372f82dd0.jpg
  Copied: 786_png.rf.48f568cb9e3787b54181bb3375ac7fdc.jpg -> val/ds1_786_png.rf.48f568cb9e3787b54181bb3375ac7fdc.jpg
  Copied: 999_png.rf.33a16b6c9b864ba5f927ef8b3f0bdd97.jpg -> val/ds1_999_png.rf.33a16b6c9b864ba5f927ef8b3f0bdd97.jpg
  Copied: 933_png.rf.c9ff390add154f054cc05453ee23f86c.jpg ->

  Copied: 646_png.rf.5abbe4b5f66fe96b306f01cf7104ec3d.jpg -> val/ds1_646_png.rf.5abbe4b5f66fe96b306f01cf7104ec3d.jpg
  Copied: 729_png.rf.927e9763c5eaeebeaed5e66102054cf5.jpg -> val/ds1_729_png.rf.927e9763c5eaeebeaed5e66102054cf5.jpg
  Copied: 670_png.rf.b9bf04961338b27494376e22212d6e57.jpg -> val/ds1_670_png.rf.b9bf04961338b27494376e22212d6e57.jpg
  Copied: 993_png.rf.92eb94670a979f141a84f11c69dc97e1.jpg -> val/ds1_993_png.rf.92eb94670a979f141a84f11c69dc97e1.jpg
  Copied: 1018_png.rf.4634e7dc52af58053da2929196650328.jpg -> val/ds1_1018_png.rf.4634e7dc52af58053da2929196650328.jpg
  Copied: 1027_png.rf.28ba5fd2dec5fe48376c5780a1c09988.jpg -> val/ds1_1027_png.rf.28ba5fd2dec5fe48376c5780a1c09988.jpg
  Copied: 1128_png.rf.07fa111163f5dae3c05edef7061e4738.jpg -> val/ds1_1128_png.rf.07fa111163f5dae3c05edef7061e4738.jpg
  Copied: 1035_png.rf.71857a6e7bc5e064de403a45021e8ec6.jpg -> val/ds1_1035_png.rf.71857a6e7bc5e064de403a45021e8ec6.jpg
  Copied: 965_png.rf.895a02221f0d9d4d18e2683ad786b134.jp

  Copied: 938_png.rf.1729f73660d09f43932b5f79a3ba3ff5.jpg -> test/ds1_938_png.rf.1729f73660d09f43932b5f79a3ba3ff5.jpg
  Copied: 1028_png.rf.e3a07a8b62a7620a94dba1d040899076.jpg -> test/ds1_1028_png.rf.e3a07a8b62a7620a94dba1d040899076.jpg
  Copied: 994_png.rf.e2f8304b4c58201a46c23b5ed605f18c.jpg -> test/ds1_994_png.rf.e2f8304b4c58201a46c23b5ed605f18c.jpg
  Copied: 644_png.rf.925335bca722aac86de268a9ffd3d466.jpg -> test/ds1_644_png.rf.925335bca722aac86de268a9ffd3d466.jpg
  Copied: 862_png.rf.871836fc48400e4bfe3f66de648dfa90.jpg -> test/ds1_862_png.rf.871836fc48400e4bfe3f66de648dfa90.jpg
  Copied: 692_png.rf.5700936a6c728e6c56d7c90b7e6fe3e9.jpg -> test/ds1_692_png.rf.5700936a6c728e6c56d7c90b7e6fe3e9.jpg
  Copied: 702_png.rf.4629d2fe4df152de3ccc32c5fa9dd210.jpg -> test/ds1_702_png.rf.4629d2fe4df152de3ccc32c5fa9dd210.jpg
  Copied: 701_png.rf.88b18e21b862d24f2ff45cb116fcf554.jpg -> test/ds1_701_png.rf.88b18e21b862d24f2ff45cb116fcf554.jpg
  Copied: 1059_png.rf.551517eb04ea213b0f906d745b2933bc